In [1]:
import requests as r
import json
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
import random
import time
import pandas as pd
import re
import numpy as np
import csv

In [2]:
# Начальный URL для дальнейшего парсинга
# URL = 'https://auto.ru/cars/used/' 
URL = 'https://auto.ru/cars/bmw/used/'
# element copied from start page to avoid flexible classes, copied: 13.10.2021
BRAND_SOURCE = "brands_soup.txt" # element copied from DT

pd.set_option('display.max_columns', None)

In [3]:
HEADERS = {
    'Connection': 'keep-alive',
    'sec-ch-ua': '"Chromium";v="94", "Google Chrome";v="94", ";Not A Brand";v="99"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"macOS"',
    'Upgrade-Insecure-Requests': '1',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.71 Safari/537.36',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'Sec-Fetch-Site': 'same-origin',
    'Sec-Fetch-Mode': 'navigate',
    'Sec-Fetch-User': '?1',
    'Sec-Fetch-Dest': 'document',
    'Referer': 'https://auto.ru/cars/used/',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'ru'
}

cookies = dict({
                'autoru_gdpr':'1',
                'suid':'d623ff3ea8568230f7365727199435bb.f86fd98831fdbc8f7cc83e6bc734fb90',
                'yandexuid':'625342111625395404',
                'counter_ga_all7':'2',
                '_csrf_token':'cbf3d961f93743c21f8b4a70feedb03997ae9354dfc79bae',
                'autoru_sid':'a%3Ag615dfe3a2oedoori3cl0hbgl2n5vbn0.9d2cb71f02ff80ffb82c172756d4884f%7C1633549882248.604800.EZNqR3FCBHVvD6N4ivq4sw.iR14jacJziqqnOk2TDS-6h8U9C5bJMUlo4wsdbtScio',
                'autoruuid':'g615dfe3a2oedoori3cl0hbgl2n5vbn0.9d2cb71f02ff80ffb82c172756d4884f',
                'yuidlt':'1',
                'bltsr':'1', 
                'X-Vertis-DC':'sas',
                'autoru-visits-count':'18',
                'autoru-visits-session-unexpired':'1',
                'from_lifetime':'1633641453855',
                'from':'direct',
                'los':'1'
                })


In [30]:
def from_lifetime(resp):
    # needed for coockies
    lst = resp.request.headers['Cookie'].split('; ')
    dict1 = {}
    for elem in lst:
        key, value = elem.split('=')
        dict1[key] = value
        return dict1['from_lifetime']
    

def get_start_pages(file_name):
    # creates dictionary {brand : start_page}
    brand_start_page = {}
    with open(file_name) as f:
        text = f.read()
    page = BeautifulSoup(text, 'html.parser')
    items = page.findAll('div', attrs={'class':'ListingPopularMMM__item'})
    for item in items:
        brand_start_page[str(item.find('a').text).upper()] = item.find('a')["href"]
        
    return brand_start_page


def get_number(url):
    # receives number of advertizements from button "Показать..n . предложений"
    # on start page for a brand
    response = r.get(url)
    response.encoding = 'utf8'
    page = BeautifulSoup(response.text, 'html.parser')
    context = page.findAll('span',{"class": "ButtonWithLoader__content"})
    result = int(re.search(r'\d+', context[0].text.replace("\xa0", "")).group(0))
    return result


def get_ad_numbers(dict_urls):
# creates a dictionary {brand :count} - number of ads for every brand
# reads numbers from start pages, button "Показать..n . предложений"
    dict_ad_counts = {}
    for brand, url in dict_urls.items():
        try:
            dict_ad_counts[brand.upper()] = get_number(url)
            print(brand, dict_ad_counts[brand])
        except Exception as ex:
            print(brand, ex)
            
    return dict_ad_counts


def save_links_to_csv(brand, dict_brand_links, dict_brands):
#  collects ad-content links for one brand
# saves them to file: "brandname_links.csv"

    URL = dict_brand_links[brand]
    
    print("brand: " + brand)
    total = str(dict_brands[brand])
    print("number of ads: " + total)
    pages_url_list = []
    df = pd.DataFrame(list())
   # df.to_csv('links.csv')
    link_counter = 0
    page_counter = 0
    
    while (link_counter <= dict_brands[brand]) and (page_counter<100):
        print("page " + str(page_counter) + " being parsed...")
        links = []
        response = r.get(URL +'?page=%s'%page_counter) 
        page = BeautifulSoup(response.text, 'html.parser')
        links = page.findAll('a',{"class": "Link OfferThumb"})
        print(str(len(links)) + " ads found")
        # print("page " + str(page_counter) + ": " + str(len(links)) + " advertisements, total: " + str(link_counter))
        for link in links:

            try:
                # if link['href'] not in pages_url_list: 
                pages_url_list.append(link['href'])
            except:
                pass
        df = pd.DataFrame(pages_url_list)
        df.to_csv('LINKS/%s_links.csv'%brand, mode='a', header=False, index=False)
        link_counter+=len(pages_url_list)
        print(str(len(pages_url_list)) + " ads added, total: " + str(link_counter) + " of " + total)   
        pages_url_list = []
        page_counter+=1
        # ждем random секунд до следующего запроса
        # rand_time = random.randint(2,4)
        # time.sleep(rand_time)
    print ("="*50)  

    
def parse_links(brands, d_brand_links, d_brand_counts):
# main function, works relativel long
# create a set of separate files for every brand: "brandname_links.csv"
    brand_OK = []
    brand_notOK = brands
    for brand in brands:
        try:
            save_links_to_csv(brand, d_brand_links, d_brand_counts)
            brand_OK.append(brand)
        except Exception as ex:
            brand_notOK.append(brand) 
            print(ex)
            pass
    remained_brands = list(set(brands) - set(brand_OK))
    return remained_brands
        


In [19]:
# get list of brands to be parsed from test dataset:
brands = list(pd.read_csv("test.csv").brand.unique())
brands

['SKODA',
 'AUDI',
 'HONDA',
 'VOLVO',
 'BMW',
 'NISSAN',
 'INFINITI',
 'MERCEDES',
 'TOYOTA',
 'LEXUS',
 'VOLKSWAGEN',
 'MITSUBISHI']

In [6]:
# I would like to use 2 dictionaries: 
# {brand : start_url} - to let function know where to start
# {brand : ads_count} - to let function stop when all ads are parsed
# these dictionaries will be saved into files to be re-used


In [29]:
# {brand : start_url}:
# create dictionary of start pages:
# {'AC': 'https://auto.ru/cars/ac/used/',
#  'Acura': 'https://auto.ru/cars/acura/used/',
#  'Adler': 'https://auto.ru/cars/adler/used/',
#  'Alfa Romeo': 'https://auto.ru/cars/alfa_romeo/used/',...,}

dict_startpages = get_start_pages(BRAND_SOURCE)
# dict_startpages
# save to file 'startpages.json':
with open('startpages.json', 'w', encoding='utf8') as json_file:
    json.dump(dict_startpages, json_file, ensure_ascii=False)
    

In [31]:
# {brand : ads_count}
# create dictionary of numbers of ads
dict_adscounts = get_ad_numbers(dict_startpages)

# save to file:
with open('ads_counts.json', 'w', encoding='utf8') as json_file:
    json.dump(dict_adscounts, json_file, ensure_ascii=False)

# dict_adscounts    

AC 2
ACURA 130
ADLER 4
ALFA ROMEO 150
ALPINA 9
AMC 2
ARIEL 1
ARO 5
ASIA 8
ASTON MARTIN 16
AUBURN 1
AUDI 7865
BAIC 3
BENTLEY 165
BMW 12433
BORGWARD 1
BRABUS 1
BRILLIANCE 69
BUICK 36
BYD 95
CADILLAC 555
CHANA 1
CHANGAN 90
CHANGFENG 3
CHERY 1720
CHERYEXEED 10
CHEVROLET 9852
CHRYSLER 605
CITROEN 1888
DACIA 44
DADI 1
DAEWOO 3068
DAIHATSU 337
DAIMLER 3
DATSUN 817
DELAGE 1
DERWAYS 23
DKW 1
DODGE 762
DONGFENG 43
DONINVEST 13
DS 1
DW HOWER 9
EAGLE 4
FAW 120
FERRARI 28
FIAT 688
FISKER 1
FLANKER 1
FORD 10773
FOTON 10
FSO 2
FUQI 1
GAC 2
GEELY 1053
GENESIS 137
GEO 4
GMC 49
GREAT WALL 835
HAFEI 30
HAIMA 60
HANOMAG 1
HAVAL 275
HAWTAI 11
HONDA 4895
HUANGHAI 3
HUDSON 1
HUMMER 171
HYUNDAI 14646
INFINITI 1560
IRAN KHODRO 85
ISUZU 114
JAC 42
JAGUAR 643
JEEP 935
JINBEI 5
KIA 14824
LADA (ВАЗ) 43548
LAMBORGHINI 16
LANCIA 31
LAND ROVER 3017
LANDWIND 1
LEXUS 3316
LIFAN 1375
LIGIER 1
LINCOLN 181
LOTUS 5
LUXGEN 11
MAHINDRA 2
MASERATI 77
MAYBACH 18
MAZDA 5909
MCLAREN 4
MERCEDES-BENZ 13563
MERCURY 35
METROCAB 2
MG

In [32]:
# read data from file startpages.json to brand_to_startpage dictionary:

try:
    with open('startpages.json') as json_file:
        brand_to_startpage = json.load(json_file)
    print("file brand_counts.json exists in folder, reading data... ")
except:
    print("file brand_counts.json DOES NOT exist in folder, please re-create by calling get_start_pages function ")

brand_to_startpage

# brands = d_brand_counts.keys()
# brands

file brand_counts.json exists in folder, reading data... 


{'AC': 'https://auto.ru/cars/ac/used/',
 'ACURA': 'https://auto.ru/cars/acura/used/',
 'ADLER': 'https://auto.ru/cars/adler/used/',
 'ALFA ROMEO': 'https://auto.ru/cars/alfa_romeo/used/',
 'ALPINA': 'https://auto.ru/cars/alpina/used/',
 'AMC': 'https://auto.ru/cars/amc/used/',
 'ARIEL': 'https://auto.ru/cars/ariel/used/',
 'ARO': 'https://auto.ru/cars/aro/used/',
 'ASIA': 'https://auto.ru/cars/asia/used/',
 'ASTON MARTIN': 'https://auto.ru/cars/aston_martin/used/',
 'AUBURN': 'https://auto.ru/cars/auburn/used/',
 'AUDI': 'https://auto.ru/cars/audi/used/',
 'BAIC': 'https://auto.ru/cars/baic/used/',
 'BENTLEY': 'https://auto.ru/cars/bentley/used/',
 'BMW': 'https://auto.ru/cars/bmw/used/',
 'BORGWARD': 'https://auto.ru/cars/borgward/used/',
 'BRABUS': 'https://auto.ru/cars/brabus/used/',
 'BRILLIANCE': 'https://auto.ru/cars/brilliance/used/',
 'BUICK': 'https://auto.ru/cars/buick/used/',
 'BYD': 'https://auto.ru/cars/byd/used/',
 'CADILLAC': 'https://auto.ru/cars/cadillac/used/',
 'CHAN

In [33]:
# read data from file ads_counts.json to brand_to_adsnumner dictionary:

try:
    with open('ads_counts.json') as json_file:
        brand_to_ads_counts = json.load(json_file)
    print("file ads_counts.json exists in folder, reading data... ")
except:
    print("file brand_counts.json DOES NOT exist in folder, please re-create by calling get_start_pages function ")

brand_to_ads_counts



file ads_counts.json exists in folder, reading data... 


{'AC': 2,
 'ACURA': 130,
 'ADLER': 4,
 'ALFA ROMEO': 150,
 'ALPINA': 9,
 'AMC': 2,
 'ARIEL': 1,
 'ARO': 5,
 'ASIA': 8,
 'ASTON MARTIN': 16,
 'AUBURN': 1,
 'AUDI': 7865,
 'BAIC': 3,
 'BENTLEY': 165,
 'BMW': 12433,
 'BORGWARD': 1,
 'BRABUS': 1,
 'BRILLIANCE': 69,
 'BUICK': 36,
 'BYD': 95,
 'CADILLAC': 555,
 'CHANA': 1,
 'CHANGAN': 90,
 'CHANGFENG': 3,
 'CHERY': 1720,
 'CHERYEXEED': 10,
 'CHEVROLET': 9852,
 'CHRYSLER': 605,
 'CITROEN': 1888,
 'DACIA': 44,
 'DADI': 1,
 'DAEWOO': 3068,
 'DAIHATSU': 337,
 'DAIMLER': 3,
 'DATSUN': 817,
 'DELAGE': 1,
 'DERWAYS': 23,
 'DKW': 1,
 'DODGE': 762,
 'DONGFENG': 43,
 'DONINVEST': 13,
 'DS': 1,
 'DW HOWER': 9,
 'EAGLE': 4,
 'FAW': 120,
 'FERRARI': 28,
 'FIAT': 688,
 'FISKER': 1,
 'FLANKER': 1,
 'FORD': 10773,
 'FOTON': 10,
 'FSO': 2,
 'FUQI': 1,
 'GAC': 2,
 'GEELY': 1053,
 'GENESIS': 137,
 'GEO': 4,
 'GMC': 49,
 'GREAT WALL': 835,
 'HAFEI': 30,
 'HAIMA': 60,
 'HANOMAG': 1,
 'HAVAL': 275,
 'HAWTAI': 11,
 'HONDA': 4895,
 'HUANGHAI': 3,
 'HUDSON': 1,
 'HU

In [34]:
#for all brands uncomment:
# brands = list(brand_to_ads_counts.keys())

brands = list(pd.read_csv("test.csv").brand.unique())


In [35]:
brands

['SKODA',
 'AUDI',
 'HONDA',
 'VOLVO',
 'BMW',
 'NISSAN',
 'INFINITI',
 'MERCEDES',
 'TOYOTA',
 'LEXUS',
 'VOLKSWAGEN',
 'MITSUBISHI']

In [37]:
# >>>>>>>> PARSING LINKS:
# creating list of files "brandname_links.csv" to store links to ads¶


for brand in brands:
    print(brand)
    save_links_to_csv(brand, brand_to_startpage, brand_to_ads_counts)
            
        

SKODA
brand: SKODA
number of ads: 6658
page 0 being parsed...
35 ads found
35 ads added, total: 35 of 6658
page 1 being parsed...
37 ads found
37 ads added, total: 72 of 6658
page 2 being parsed...
37 ads found
37 ads added, total: 109 of 6658
page 3 being parsed...
38 ads found
38 ads added, total: 147 of 6658
page 4 being parsed...
37 ads found
37 ads added, total: 184 of 6658
page 5 being parsed...
36 ads found
36 ads added, total: 220 of 6658
page 6 being parsed...
38 ads found
38 ads added, total: 258 of 6658
page 7 being parsed...
38 ads found
38 ads added, total: 296 of 6658
page 8 being parsed...
38 ads found
38 ads added, total: 334 of 6658
page 9 being parsed...
38 ads found
38 ads added, total: 372 of 6658
page 10 being parsed...
38 ads found
38 ads added, total: 410 of 6658
page 11 being parsed...
33 ads found
33 ads added, total: 443 of 6658
page 12 being parsed...
36 ads found
36 ads added, total: 479 of 6658
page 13 being parsed...
27 ads found
27 ads added, total: 506 o

36 ads found
36 ads added, total: 568 of 7865
page 16 being parsed...
32 ads found
32 ads added, total: 600 of 7865
page 17 being parsed...
28 ads found
28 ads added, total: 628 of 7865
page 18 being parsed...
34 ads found
34 ads added, total: 662 of 7865
page 19 being parsed...
36 ads found
36 ads added, total: 698 of 7865
page 20 being parsed...
37 ads found
37 ads added, total: 735 of 7865
page 21 being parsed...
37 ads found
37 ads added, total: 772 of 7865
page 22 being parsed...
36 ads found
36 ads added, total: 808 of 7865
page 23 being parsed...
37 ads found
37 ads added, total: 845 of 7865
page 24 being parsed...
37 ads found
37 ads added, total: 882 of 7865
page 25 being parsed...
37 ads found
37 ads added, total: 919 of 7865
page 26 being parsed...
37 ads found
37 ads added, total: 956 of 7865
page 27 being parsed...
37 ads found
37 ads added, total: 993 of 7865
page 28 being parsed...
37 ads found
37 ads added, total: 1030 of 7865
page 29 being parsed...
37 ads found
37 ads

37 ads found
37 ads added, total: 1149 of 4895
page 31 being parsed...
37 ads found
37 ads added, total: 1186 of 4895
page 32 being parsed...
37 ads found
37 ads added, total: 1223 of 4895
page 33 being parsed...
37 ads found
37 ads added, total: 1260 of 4895
page 34 being parsed...
37 ads found
37 ads added, total: 1297 of 4895
page 35 being parsed...
37 ads found
37 ads added, total: 1334 of 4895
page 36 being parsed...
37 ads found
37 ads added, total: 1371 of 4895
page 37 being parsed...
37 ads found
37 ads added, total: 1408 of 4895
page 38 being parsed...
37 ads found
37 ads added, total: 1445 of 4895
page 39 being parsed...
37 ads found
37 ads added, total: 1482 of 4895
page 40 being parsed...
37 ads found
37 ads added, total: 1519 of 4895
page 41 being parsed...
37 ads found
37 ads added, total: 1556 of 4895
page 42 being parsed...
37 ads found
37 ads added, total: 1593 of 4895
page 43 being parsed...
37 ads found
37 ads added, total: 1630 of 4895
page 44 being parsed...
37 ads

37 ads found
37 ads added, total: 1670 of 2671
page 46 being parsed...
37 ads found
37 ads added, total: 1707 of 2671
page 47 being parsed...
37 ads found
37 ads added, total: 1744 of 2671
page 48 being parsed...
37 ads found
37 ads added, total: 1781 of 2671
page 49 being parsed...
37 ads found
37 ads added, total: 1818 of 2671
page 50 being parsed...
37 ads found
37 ads added, total: 1855 of 2671
page 51 being parsed...
37 ads found
37 ads added, total: 1892 of 2671
page 52 being parsed...
37 ads found
37 ads added, total: 1929 of 2671
page 53 being parsed...
37 ads found
37 ads added, total: 1966 of 2671
page 54 being parsed...
37 ads found
37 ads added, total: 2003 of 2671
page 55 being parsed...
37 ads found
37 ads added, total: 2040 of 2671
page 56 being parsed...
37 ads found
37 ads added, total: 2077 of 2671
page 57 being parsed...
37 ads found
37 ads added, total: 2114 of 2671
page 58 being parsed...
37 ads found
37 ads added, total: 2151 of 2671
page 59 being parsed...
37 ads

37 ads found
37 ads added, total: 3030 of 12433
page 86 being parsed...
37 ads found
37 ads added, total: 3067 of 12433
page 87 being parsed...
37 ads found
37 ads added, total: 3104 of 12433
page 88 being parsed...
37 ads found
37 ads added, total: 3141 of 12433
page 89 being parsed...
37 ads found
37 ads added, total: 3178 of 12433
page 90 being parsed...
37 ads found
37 ads added, total: 3215 of 12433
page 91 being parsed...
37 ads found
37 ads added, total: 3252 of 12433
page 92 being parsed...
37 ads found
37 ads added, total: 3289 of 12433
page 93 being parsed...
37 ads found
37 ads added, total: 3326 of 12433
page 94 being parsed...
37 ads found
37 ads added, total: 3363 of 12433
page 95 being parsed...
37 ads found
37 ads added, total: 3400 of 12433
page 96 being parsed...
37 ads found
37 ads added, total: 3437 of 12433
page 97 being parsed...
37 ads found
37 ads added, total: 3474 of 12433
page 98 being parsed...
37 ads found
37 ads added, total: 3511 of 12433
page 99 being pa

37 ads found
37 ads added, total: 3635 of 13943
INFINITI
brand: INFINITI
number of ads: 1560
page 0 being parsed...
36 ads found
36 ads added, total: 36 of 1560
page 1 being parsed...
33 ads found
33 ads added, total: 69 of 1560
page 2 being parsed...
37 ads found
37 ads added, total: 106 of 1560
page 3 being parsed...
32 ads found
32 ads added, total: 138 of 1560
page 4 being parsed...
32 ads found
32 ads added, total: 170 of 1560
page 5 being parsed...
34 ads found
34 ads added, total: 204 of 1560
page 6 being parsed...
37 ads found
37 ads added, total: 241 of 1560
page 7 being parsed...
37 ads found
37 ads added, total: 278 of 1560
page 8 being parsed...
37 ads found
37 ads added, total: 315 of 1560
page 9 being parsed...
37 ads found
37 ads added, total: 352 of 1560
page 10 being parsed...
37 ads found
37 ads added, total: 389 of 1560
page 11 being parsed...
37 ads found
37 ads added, total: 426 of 1560
page 12 being parsed...
37 ads found
37 ads added, total: 463 of 1560
page 13 b

KeyError: 'MERCEDES'

In [38]:
brands = ["MERCEDES-BENZ","TOYOTA", "LEXUS", "VOLKSWAGEN", "MITSUBISHI"]


In [39]:
for brand in brands:
    print(brand)
    save_links_to_csv(brand, brand_to_startpage, brand_to_ads_counts)
            

MERCEDES-BENZ
brand: MERCEDES-BENZ
number of ads: 13563
page 0 being parsed...
33 ads found
33 ads added, total: 33 of 13563
page 1 being parsed...
33 ads found
33 ads added, total: 66 of 13563
page 2 being parsed...
16 ads found
16 ads added, total: 82 of 13563
page 3 being parsed...
34 ads found
34 ads added, total: 116 of 13563
page 4 being parsed...
36 ads found
36 ads added, total: 152 of 13563
page 5 being parsed...
37 ads found
37 ads added, total: 189 of 13563
page 6 being parsed...
37 ads found
37 ads added, total: 226 of 13563
page 7 being parsed...
38 ads found
38 ads added, total: 264 of 13563
page 8 being parsed...
38 ads found
38 ads added, total: 302 of 13563
page 9 being parsed...
37 ads found
37 ads added, total: 339 of 13563
page 10 being parsed...
28 ads found
28 ads added, total: 367 of 13563
page 11 being parsed...
15 ads found
15 ads added, total: 382 of 13563
page 12 being parsed...
12 ads found
12 ads added, total: 394 of 13563
page 13 being parsed...
24 ads fou

19 ads found
19 ads added, total: 430 of 19215
page 14 being parsed...
15 ads found
15 ads added, total: 445 of 19215
page 15 being parsed...
29 ads found
29 ads added, total: 474 of 19215
page 16 being parsed...
33 ads found
33 ads added, total: 507 of 19215
page 17 being parsed...
36 ads found
36 ads added, total: 543 of 19215
page 18 being parsed...
38 ads found
38 ads added, total: 581 of 19215
page 19 being parsed...
37 ads found
37 ads added, total: 618 of 19215
page 20 being parsed...
37 ads found
37 ads added, total: 655 of 19215
page 21 being parsed...
34 ads found
34 ads added, total: 689 of 19215
page 22 being parsed...
38 ads found
38 ads added, total: 727 of 19215
page 23 being parsed...
38 ads found
38 ads added, total: 765 of 19215
page 24 being parsed...
38 ads found
38 ads added, total: 803 of 19215
page 25 being parsed...
38 ads found
38 ads added, total: 841 of 19215
page 26 being parsed...
38 ads found
38 ads added, total: 879 of 19215
page 27 being parsed...
37 ads

37 ads found
37 ads added, total: 1001 of 3316
page 28 being parsed...
37 ads found
37 ads added, total: 1038 of 3316
page 29 being parsed...
37 ads found
37 ads added, total: 1075 of 3316
page 30 being parsed...
37 ads found
37 ads added, total: 1112 of 3316
page 31 being parsed...
37 ads found
37 ads added, total: 1149 of 3316
page 32 being parsed...
37 ads found
37 ads added, total: 1186 of 3316
page 33 being parsed...
37 ads found
37 ads added, total: 1223 of 3316
page 34 being parsed...
37 ads found
37 ads added, total: 1260 of 3316
page 35 being parsed...
37 ads found
37 ads added, total: 1297 of 3316
page 36 being parsed...
37 ads found
37 ads added, total: 1334 of 3316
page 37 being parsed...
37 ads found
37 ads added, total: 1371 of 3316
page 38 being parsed...
37 ads found
37 ads added, total: 1408 of 3316
page 39 being parsed...
37 ads found
37 ads added, total: 1445 of 3316
page 40 being parsed...
37 ads found
37 ads added, total: 1482 of 3316
page 41 being parsed...
37 ads

37 ads found
37 ads added, total: 1539 of 14571
page 43 being parsed...
37 ads found
37 ads added, total: 1576 of 14571
page 44 being parsed...
36 ads found
36 ads added, total: 1612 of 14571
page 45 being parsed...
37 ads found
37 ads added, total: 1649 of 14571
page 46 being parsed...
37 ads found
37 ads added, total: 1686 of 14571
page 47 being parsed...
37 ads found
37 ads added, total: 1723 of 14571
page 48 being parsed...
37 ads found
37 ads added, total: 1760 of 14571
page 49 being parsed...
37 ads found
37 ads added, total: 1797 of 14571
page 50 being parsed...
37 ads found
37 ads added, total: 1834 of 14571
page 51 being parsed...
37 ads found
37 ads added, total: 1871 of 14571
page 52 being parsed...
37 ads found
37 ads added, total: 1908 of 14571
page 53 being parsed...
37 ads found
37 ads added, total: 1945 of 14571
page 54 being parsed...
37 ads found
37 ads added, total: 1982 of 14571
page 55 being parsed...
37 ads found
37 ads added, total: 2019 of 14571
page 56 being pa

36 ads found
36 ads added, total: 2073 of 8354
page 57 being parsed...
37 ads found
37 ads added, total: 2110 of 8354
page 58 being parsed...
37 ads found
37 ads added, total: 2147 of 8354
page 59 being parsed...
37 ads found
37 ads added, total: 2184 of 8354
page 60 being parsed...
37 ads found
37 ads added, total: 2221 of 8354
page 61 being parsed...
37 ads found
37 ads added, total: 2258 of 8354
page 62 being parsed...
37 ads found
37 ads added, total: 2295 of 8354
page 63 being parsed...
37 ads found
37 ads added, total: 2332 of 8354
page 64 being parsed...
37 ads found
37 ads added, total: 2369 of 8354
page 65 being parsed...
37 ads found
37 ads added, total: 2406 of 8354
page 66 being parsed...
37 ads found
37 ads added, total: 2443 of 8354
page 67 being parsed...
37 ads found
37 ads added, total: 2480 of 8354
page 68 being parsed...
37 ads found
37 ads added, total: 2517 of 8354
page 69 being parsed...
37 ads found
37 ads added, total: 2554 of 8354
page 70 being parsed...
37 ads